In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

: 

In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
correlation_matrix = train.corr(numeric_only=True)
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title("Correlation heatmap")
plt.show()

In [ ]:
# Set correlation threshold
threshold = 0.7

# Create a mask for absolute correlations above threshold
high_corr = correlation_matrix.abs() > threshold

# Extract correlated pairs (excluding self-correlations)
correlated_pairs = []

for col in correlation_matrix.columns:
    for idx in correlation_matrix.index:
        if col != idx and high_corr.loc[idx, col]:
            correlated_pairs.append((idx, col, correlation_matrix.loc[idx, col]))

# Convert to DataFrame for easier viewing
corr_df = pd.DataFrame(correlated_pairs, columns=["Feature 1", "Feature 2", "Correlation"])

# Drop duplicate pairs (A,B) == (B,A)
corr_df = corr_df.sort_values(by="Correlation", ascending=False).drop_duplicates(subset=["Correlation"])

print("\n===== Highly Correlated Feature Pairs =====")
print(corr_df)

We created a correlation matrix of all of the numeric (non-categorial)
variables. There are 6 pairs of features with an absolute value
correlation higher than 0.7.

From the above table, we would want to choose only Feature 1 or Feature
2 to avoid high multicollinearity in our model.

In [ ]:
from scipy import stats
import numpy as np

z_scores = np.abs(stats.zscore(train.select_dtypes(include='number')))
print((z_scores > 3).sum())

In [ ]:
# Compute absolute z-scores
numeric_train = train.select_dtypes(include='number')

z_scores = np.abs(stats.zscore(numeric_train, nan_policy='omit'))

# Convert to DataFrame
z_df = pd.DataFrame(z_scores, columns=numeric_train.columns)

# Threshold
threshold = 3

# Boolean mask for outliers
outlier_mask = (z_df > threshold)

# Count outliers per column
outlier_counts = outlier_mask.sum().sort_values(ascending=False)

print("\n===== Outlier Counts per Column =====")
print(outlier_counts[outlier_counts > 0])

# 🔍 Print column names that actually have outliers
outlier_columns = outlier_counts[outlier_counts > 0].index.tolist()
print("\nColumns with outliers:")
print(outlier_columns)

In [ ]:
# # outliers
# train.hist(bins=30, figsize=(15,10))
# plt.show()

# for col in train.select_dtypes(include='number').columns:
#     sns.boxplot(x=train[col])
#     plt.title(col)
#     plt.show()

In [ ]:
# Possible features to add:
# Total finished basement square feet:
# BsmtFinSF1 + BsmtFinSF2.

# Total sqft (above and below grade)
# Total finished basement square feet + GrLivArea.

# Can remove 1stFlrSF
train['total_BsmtFin_sqft'] = train['BsmtFinSF1'] + train['BsmtFinSF2']
train['total_sqft'] = train['total_BsmtFin_sqft'] + train['GrLivArea']